# RegEx 2:

Let's use the `re` and the `biopython` packages to write scripts to search for restriction sites in yeast genes and protein motifs in the yeast proteome.

Our first step will be to download all the proteins in yeast, a file with all these proteins can b e found at SGD in the following url: http://downloads.yeastgenome.org/sequence/S288C_reference/orf_protein/orf_trans.fasta.gz. We will also download all the gene sequences from yeast: http://downloads.yeastgenome.org/sequence/S288C_reference/orf_dna/orf_coding.fasta.gz

We will use the `urllib` package to download the files:

In [ ]:
import urllib
url = "http://downloads.yeastgenome.org/sequence/S288C_reference/orf_protein/orf_trans.fasta.gz"
urllib.urlretrieve(url,'./orf_trans.fasta.gz')  #first get the webpage you want
protFile = 'orf_trans.fasta'

url = "http://downloads.yeastgenome.org/sequence/S288C_reference/orf_dna/orf_coding.fasta.gz"
urllib.urlretrieve(url,'./orf_coding.fasta.gz')  #first get the webpage you want
orfFile = 'orf_coding.fasta'

Now we will uncompress the files. Depending on your computer you might need to uncompress the file using the Finder or Windows Explorer.

In [ ]:
!gunzip orf_coding.fasta.gz
!gunzip orf_trans.fasta.gz

### Finding restriction sites in yeast genes:


#### Create a dataframe with all yeast genes: 

Let's write a script to read all the open reading frames and add them to a dataframe.

In [ ]:
orfFile = 'orf_coding.fasta'
from Bio import SeqIO
information = []
names = []
for seq_record in SeqIO.parse(orfFile, "fasta"):
    names.append(seq_record.name)
    information.append([seq_record.name ,seq_record.description, str(seq_record.seq)])

In [ ]:
import pandas as pd
allORFs = pd.DataFrame(information, columns=['acc','description','seq'])
allORFs.head()

#### Looping through a dataframe

In [ ]:
for index, row in allORFs.iterrows():
    print index
    print row
    print row.acc
    print row['acc']
    break

#### Search for restriction sites:

In [ ]:
# find all yeast genes that have an EcoRI restriction site and tell how many cuts in that gene
import re
genes = []
for index, row in allORFs.iterrows():
    result = re.findall('GAATTC',row.seq)
    if len(result) > 0:
        genes.append([row.acc, len(result)])
genesEco = pd.DataFrame(genes, columns = ['acc','ecoRI'])
genesEco.describe()

In [ ]:
import matplotlib.pyplot as plt
genesEco['ecoRI'].hist()
plt.show()

In [ ]:
genes = []
for index, row in allORFs.iterrows():
    result = re.findall('(?=(GC\wGC))',row.seq)
    if len(result) > 0:
        genes.append([row.acc, len(result)])
genesBisI = pd.DataFrame(genes, columns = ['acc','bisI'])
genesBisI.describe()

In [ ]:
import matplotlib.pyplot as plt
genesBisI['bisI'].hist()
plt.show()

#### Merging the results dataframes

In [ ]:
genesBoth = pd.merge(genesEco, genesBisI, how='outer', on ='acc')
genesBoth.describe()

In [ ]:
print genesBoth.head()
print genesBoth.tail()

In [ ]:
%matplotlib inline
genesBoth.hist()

In [ ]:
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, plot, iplot
init_notebook_mode()

In [ ]:
data1 = go.Histogram(x=genesEco.ecoRI, opacity=0.75, name='EcoRI')
data2 = go.Histogram(x=genesBisI.bisI, opacity=0.75, name='BisI')
iplot([data1,data2])

In [ ]:
layout = go.Layout(barmode='overlay')
fig = go.Figure(data=[data1,data2], layout=layout)
iplot(fig)

## Search for protein domains:

PROSITE DATABASE: http://prosite.expasy.org

PROSITE is a database of protein families and domains. It is based on the observation that, while there is a huge number of different proteins, most of them can be grouped, on the basis of similarities in their sequences, into a limited number of families. Proteins or protein domains belonging to a particular family generally share functional attributes and are derived from a common ancestor.

It is apparent, when studying protein sequence families, that some regions have been better conserved than others during evolution. These regions are generally important for the function of a protein and/or for the maintenance of its three- dimensional structure. By analyzing the constant and variable properties of such groups of similar sequences, it is possible to derive a signature for a protein family or domain, which distinguishes its members from all other unrelated proteins. A pertinent analogy is the use of fingerprints by the police for identification purposes. A fingerprint is generally sufficient to identify a given individual. Similarly, a protein signature can be used to assign a newly sequenced protein to a specific family of proteins and thus to formulate hypotheses about its function.

PROSITE currently contains patterns and profiles specific for more than a thousand protein families or domains. Each of these signatures comes with documentation providing background information on the structure and function of these proteins.


PATTERNS, FROM THE DATABASE DESCRIPTION:

The PA (PAttern) lines contains the definition of a PROSITE pattern. The patterns are described using the following conventions:

1. The standard IUPAC one-letter codes for the amino acids are used.
2. The symbol 'x' is used for a position where any amino acid is accepted.
3. Ambiguities are indicated by listing the acceptable amino acids for a given position, between square parentheses '[ ]'. For example: [ALT] stands for Ala or Leu or Thr.
4. Ambiguities are also indicated by listing between a pair of curly brackets '{ }' the amino acids that are not accepted at a given position. For example: {AM} stands for any amino acid except Ala and Met.
5. Each element in a pattern is separated from its neighbor by a '-'.
6. Repetition of an element of the pattern can be indicated by following that element with a numerical value or a numerical range between parenthesis. Examples: x(3) corresponds to x-x-x, x(2,4) corresponds to x-x or x-x-x or x-x-x-x.
7. When a pattern is restricted to either the N- or C-terminal of a sequence, that pattern either starts with a '<' symbol or respectively ends with a '>' symbol. In some rare cases (e.g. PS00267 or PS00539), '>' can also occur inside square brackets for the C-terminal element. 'F-[GSTV]-P-R-L-[G>]' means that either 'F-[GSTV]-P-R-L-G' or 'F-[GSTV]-P-R-L>' are considered.
8. A period ends the pattern.


EXAMPLES:
ER_TARGET: http://prosite.expasy.org/cgi-bin/prosite/prosite-search-ac?PDOC00014

```
[KRHQSA]-[DENQ]-E-L>.
```

In Python: 
```
[KRHQSA][DENQ]EL$
```


Hsp90 proteins: http://prosite.expasy.org/PDOC00270

```
Y-x-[NQHD]-[KHR]-[DE]-[IVA]-F-[LM]-R-[ED].
```

In Python:

```
Y\w[NQHD][KHR][DE][IVA]F[LM]R[ED]
```

In [ ]:
from Bio import SeqIO
information = []
for seq_record in SeqIO.parse(protFile, "fasta"):
    result = re.search('[KRHQSA][DENQ]EL\*',str(seq_record.seq))
    if result:
        information.append([seq_record.name ,result.group(), result.span(), str(seq_record.description)])

pd.DataFrame(information, columns=['acc','match','start_end','seq'])

In [ ]:
pattern = "Y\w[NQHD][KHR][DE][IVA]F[LM]R[ED]"

def findinseqfile(pattern,filein):
    information = []
    for seq_record in SeqIO.parse(filein, "fasta"):
        result = re.search(pattern,str(seq_record.seq))
        if result:
            information.append([seq_record.name ,result.group(), result.span(), str(seq_record.description)])

    return pd.DataFrame(information, columns=['acc','match','start_end','seq'])

findinseqfile(pattern, protFile)